<img src='https://user-images.githubusercontent.com/6457691/89746244-a6e2a200-daf3-11ea-9314-7863d544d73c.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 1 / NOTE 2*

# 📝 Assignment


## 1. Feature Engineering

- `KT&G` 기업 데이터를 계속 사용합니다.
  - 변수 이름은 `df`로 저장하세요.

- 각 분기별 영업이익률을 직접 계산해보세요. 
  - Feature 이름은 `영업이익률2` 입니다.

- 아래 데이터를 참조하여 18/19년도에 대해서 Price to Dream Ratio (PDR)을 계산하세요.
  - 변수 이름은 `df2`, feature 이름은 `PDR`로, `df2['PDR']`을 통해 `~` 값이 확인 되어야합니다. 아래의 전제조건을 활용하세요.
  - 점유율 (일반담배 기준) 
    - 18년 : 62%
    - 19년 : 63.5% 
  - 시장의 규모 
     - 18년 : 15조 
     - 19년 : 15조

||구분|종가|발행주식수|시가총액|시장점유율|PDR|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|0|18년|101500|137292497|13935188445500|62.0|~|
|1|19년|93800|137292497|12878036218600|63.5|~|


In [141]:
import pandas as pd
import datetime as dt

url = 'https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/kt%26g/kt%26g_0.csv'
headers = ['분기', '매출액', '영업이익', '영업이익(발표기준)', '세전계속사업이익',
           '당기순이익', '당기순이익(지배)', '당기순이익(비지배)', '자산총계', '부채총계',
           '자본총계', '자본총계(지배)', '자본총계(비지배)', '자본금', '영업활동현금흐름',
           '투자활동현금흐름', '재무활동현금흐름', '영업이익률', '순이익률', 'ROE(%)',
           'ROA(%)', '부채비율', '자본유보율', 'EPS(원)', 'PER(배)']

df = pd.read_csv(url, header=None)
df.columns = headers

print(df.dtypes)
# 매출액 object, 영업이익 object이므로 int로 형변환
def toInt(string):
  return int(string.replace(',', ''))

# 영업이익률 = (영업이익/매출액) * 100
df['영업이익률2'] = (df['영업이익'].apply(toInt) / df['매출액'].apply(toInt)) * 100

df

분기             object
매출액            object
영업이익           object
영업이익(발표기준)     object
세전계속사업이익       object
당기순이익          object
당기순이익(지배)      object
당기순이익(비지배)      int64
자산총계           object
부채총계           object
자본총계           object
자본총계(지배)       object
자본총계(비지배)     float64
자본금            object
영업활동현금흐름       object
투자활동현금흐름       object
재무활동현금흐름       object
영업이익률         float64
순이익률          float64
ROE(%)        float64
ROA(%)        float64
부채비율          float64
자본유보율         float64
EPS(원)         object
PER(배)        float64
dtype: object


,분기,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,자본총계(지배),자본총계(비지배),자본금,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,영업이익률,순이익률,ROE(%),ROA(%),부채비율,자본유보율,EPS(원),PER(배),영업이익률2
0,19/6,"12,578","3,991","3,991","4,452","3,245","3,228",17,"108,464","25,940","82,524","81,965",NaN,"9,550","4,641","-1,878","-4,919",31.73,25.80,12.35,9.37,31.43,803.22,"2,351",13.77,31.730005
1,19/9,"13,222","3,825","3,825","4,503","3,221","3,208",13,"106,314","20,691","85,623","85,049",NaN,"9,550","-2,278",-83,-329,28.93,24.36,12.67,9.95,24.17,836.86,"2,337",13.87,28.929058
2,19/12,"11,982","2,523","2,523","1,794","1,183","1,198",-15,"107,121","20,062","87,059","86,506",NaN,"9,550",837,-501,-129,21.06,9.87,12.39,9.96,23.04,850.82,873,12.42,21.056585
3,20/3,"11,784","3,150","3,150","4,156","2,939","2,930",9,"108,594","24,862","83,732","83,170",562.0,"9,550","2,336","-1,392",-86,26.73,24.94,13.05,9.79,29.69,823.16,"2,134",9.71,26.731161
4,20/6,"13,188","3,947","3,947","4,020","2,935","2,931",4,"110,282","23,386","86,896","86,330",566.0,"9,550","7,855",-214,"-5,501",29.93,22.25,12.20,9.40,26.91,837.40,"2,135",10.46,29.928723


In [102]:
headers = ['구분', '종가', '발행주식수', '시가총액', '시장점유율']
data = [['18년', 101500, 137292497, 13935188445500, 62.0],
        ['19년', 93800, 137292497, 12878036218600, 63.5]]

df2 = pd.DataFrame(data)
df2.columns = headers

# PDR = (시가총액 / (시장규모 * 점유율)) * 100
tam = 15000000000000
df2['PDR'] = (df2['시가총액'] / (tam * df2['시장점유율']/100)) * 100
df2

,구분,종가,발행주식수,시가총액,시장점유율,PDR
0,18년,101500,137292497,13935188445500,62.0,149.840736
1,19년,93800,137292497,12878036218600,63.5,135.202480


## 2. String Manipulation

- replace가 아닌 다른 방법을 사용하여 19년도 2분기의 매출액 을 integer로 변경하세요. 결과값은 12578이 나와야합니다.
  


In [104]:
import re
pd.set_option('mode.chained_assignment',  None)

# ','를 대체시킬 함수
def comma_num_to_int(text):
  # 들어온 값의 타입을 확인하여 str이면 대체, 아니면 원래 값으로 돌려줌
  if str(type(text)) == "<class 'str'>":
    n = re.sub(",", "", text)
    return int(n)
  else:
    return text

target = df.index[(df['분기'] == '19/6')] # 19년도 2분기 행 위치 찾기
df['매출액'][target[0]] = comma_num_to_int(df.loc[target[0]]['매출액']) # 값 타입을 int로 바꾸기
df['매출액'][target[0]]

12578

## 3. Apply
- 데이터셋의 분기를 제외한 모든 feature에 대해서 수치형 데이터 (`int`, `float`) 로 수정하고, `describe`를 통해 summary statistics 를 확인하세요.
- 단 결측치는 `0`으로 처리합니다.

In [105]:
# 각 컬럼별 타입 확인
df.dtypes

# 결측치 처리
df.isna()
df = df.fillna(0)

# '분기'와 '분기' 외 컬럼을 분리
df_object = df.iloc[:, 0]
df_numeric = df.iloc[:, 1:]

# '분기' 제외 나머지 feature에 대해서 수치형 데이터로 수정
df_numeric = df_numeric.replace(',', '', regex=True).apply(pd.to_numeric)

# 분리했던 데이터프레임 합치기
df = pd.concat([df_object,df_numeric], axis=1)

df.dtypes

분기             object
매출액             int64
영업이익            int64
영업이익(발표기준)      int64
세전계속사업이익        int64
당기순이익           int64
당기순이익(지배)       int64
당기순이익(비지배)      int64
자산총계            int64
부채총계            int64
자본총계            int64
자본총계(지배)        int64
자본총계(비지배)     float64
자본금             int64
영업활동현금흐름        int64
투자활동현금흐름        int64
재무활동현금흐름        int64
영업이익률         float64
순이익률          float64
ROE(%)        float64
ROA(%)        float64
부채비율          float64
자본유보율         float64
EPS(원)          int64
PER(배)        float64
영업이익률2        float64
dtype: object

In [106]:
# summary statistics 확인
df.describe()

,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,자본총계(지배),자본총계(비지배),자본금,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,영업이익률,순이익률,ROE(%),ROA(%),부채비율,자본유보율,EPS(원),PER(배),영업이익률2
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.00000,5.000000,5.00000,5.0,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000
mean,12550.800000,3487.200000,3487.200000,3785.000000,2704.600000,2699.000000,5.600000,108155.000000,22988.20000,85166.80000,84604.000000,225.60000,9550.0,2678.200000,-813.600000,-2192.80000,27.676000,21.444000,12.532000,9.694000,27.048000,830.29200,1966.000000,12.046000,27.675106
std,664.979097,636.931079,636.931079,1131.079131,863.422724,851.338358,12.481987,1522.396795,2560.38419,1988.82596,1988.427897,308.91876,0.0,3831.542731,783.981696,2763.51139,4.115651,6.601267,0.335738,0.290224,3.554859,18.01957,619.939513,1.898112,4.116711
min,11784.000000,2523.000000,2523.000000,1794.000000,1183.000000,1198.000000,-15.000000,106314.000000,20062.00000,82524.00000,81965.000000,0.00000,9550.0,-2278.000000,-1878.000000,-5501.00000,21.060000,9.870000,12.200000,9.370000,23.040000,803.22000,873.000000,9.710000,21.056585
25%,11982.000000,3150.000000,3150.000000,4020.000000,2935.000000,2930.000000,4.000000,107121.000000,20691.00000,83732.00000,83170.000000,0.00000,9550.0,837.000000,-1392.000000,-4919.00000,26.730000,22.250000,12.350000,9.400000,24.170000,823.16000,2134.000000,10.460000,26.731161
50%,12578.000000,3825.000000,3825.000000,4156.000000,2939.000000,2931.000000,9.000000,108464.000000,23386.00000,85623.00000,85049.000000,0.00000,9550.0,2336.000000,-501.000000,-329.00000,28.930000,24.360000,12.390000,9.790000,26.910000,836.86000,2135.000000,12.420000,28.929058
75%,13188.000000,3947.000000,3947.000000,4452.000000,3221.000000,3208.000000,13.000000,108594.000000,24862.00000,86896.00000,86330.000000,562.00000,9550.0,4641.000000,-214.000000,-129.00000,29.930000,24.940000,12.670000,9.950000,29.690000,837.40000,2337.000000,13.770000,29.928723
max,13222.000000,3991.000000,3991.000000,4503.000000,3245.000000,3228.000000,17.000000,110282.000000,25940.00000,87059.00000,86506.000000,566.00000,9550.0,7855.000000,-83.000000,-86.00000,31.730000,25.800000,13.050000,9.960000,31.430000,850.82000,2351.000000,13.870000,31.730005


## 🔥 도전과제



### NA Value Handling

- 19년도 4분기의 당기순이익(비지배) 부분을 Na로 대체하세요
- 이후 해당 결측치를 mean imputation 방법을 사용하여 처리하세요.



In [124]:
import numpy as np

# 19년도 4분기(19/12) 당기순이익(비지배) 위치 찾기
row = df.index[(df['분기'] == '19/12')][0]
col = df.columns.get_loc('당기순이익(비지배)')

# 19년도 4분기(19/12) 당기순이익(비지배) Na 대체하기
df.iloc[row, col] = np.nan
print(df.iloc[row, col])

# 해당 결측치 mean imputation 처리
mean = df['당기순이익(비지배)'].mean()
print(mean)
df['당기순이익(비지배)'].fillna(mean, inplace=True)

df

nan
10.75


,분기,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,자본총계(지배),자본총계(비지배),자본금,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,영업이익률,순이익률,ROE(%),ROA(%),부채비율,자본유보율,EPS(원),PER(배),영업이익률2
0,19/6,12578,3991,3991,4452,3245,3228,17.00,108464,25940,82524,81965,0.0,9550,4641,-1878,-4919,31.73,25.80,12.35,9.37,31.43,803.22,2351,13.77,31.730005
1,19/9,13222,3825,3825,4503,3221,3208,13.00,106314,20691,85623,85049,0.0,9550,-2278,-83,-329,28.93,24.36,12.67,9.95,24.17,836.86,2337,13.87,28.929058
2,19/12,11982,2523,2523,1794,1183,1198,10.75,107121,20062,87059,86506,0.0,9550,837,-501,-129,21.06,9.87,12.39,9.96,23.04,850.82,873,12.42,21.056585
3,20/3,11784,3150,3150,4156,2939,2930,9.00,108594,24862,83732,83170,562.0,9550,2336,-1392,-86,26.73,24.94,13.05,9.79,29.69,823.16,2134,9.71,26.731161
4,20/6,13188,3947,3947,4020,2935,2931,4.00,110282,23386,86896,86330,566.0,9550,7855,-214,-5501,29.93,22.25,12.20,9.40,26.91,837.40,2135,10.46,29.928723


### Feature Engineering

Relative Perfomance 라는 새로운 feature를 계산하세요.

이는 최근 1년치 매출액의 평균값을 기준으로 

- 10% 이상 -> `S`
- 5% 이상 -> `A`
- -5 ~ 5% -> `B`
- -5%이하 -> `C`
- -10%이하 -> `D`

라는 값을 갖는 feature입니다.

20년도 2분기에 해당하는 결과값은 `A`가 나와야합니다.

각각에 해당하는 등급이 나오기 위해서 필요한 매출액을 추가로 서술하세요.

In [140]:
sales_mean = df['매출액'].mean()
print(f'매출액 평균 : {sales_mean}')

# 조건에 따라 Relative Perfomance의 값 보내주기
def getRP(sales):
  if sales >= sales_mean*1.1: # 10% 이상
    return 'S'
  elif sales >= sales_mean*1.05: # 5% 이상
    return 'A'
  elif (sales < sales_mean*1.05) & (sales > sales_mean*0.95): # -5 ~ 5%
    return 'B'
  elif sales <= sales_mean*0.95: # -5% 이하
    return 'C'
  elif sales <= sales_mean*0.9: # -10% 이하
    return 'D'
  else:
    print('getRP error')

# 'Relative Perfomance' feature 만들기
df['Relative Perfomance'] = df['매출액'].apply(getRP)

# 등급 조건 서술
print(f'S등급 매출액 조건 : {round(sales_mean*1.1, 1)} 이상')
print(f'A등급 매출액 조건 : {round(sales_mean*1.05, 1)} 이상')
print(f'B등급 매출액 조건 : {round(sales_mean*0.95, 1)} ~ {round(sales_mean*1.05, 1)}')
print(f'C등급 매출액 조건 : {round(sales_mean*0.95, 1)} 이하')
print(f'D등급 매출액 조건 : {round(sales_mean*0.9, 1)} 이하')

df

매출액 평균 : 12550.8
S등급 매출액 조건 : 13805.9 이상
A등급 매출액 조건 : 13178.3 이상
B등급 매출액 조건 : 11923.3 ~ 13178.3
C등급 매출액 조건 : 11923.3 이하
D등급 매출액 조건 : 11295.7 이하


,분기,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,자본총계(지배),자본총계(비지배),자본금,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,영업이익률,순이익률,ROE(%),ROA(%),부채비율,자본유보율,EPS(원),PER(배),영업이익률2,Relative Perfomance
0,19/6,12578,3991,3991,4452,3245,3228,17.00,108464,25940,82524,81965,0.0,9550,4641,-1878,-4919,31.73,25.80,12.35,9.37,31.43,803.22,2351,13.77,31.730005,B
1,19/9,13222,3825,3825,4503,3221,3208,13.00,106314,20691,85623,85049,0.0,9550,-2278,-83,-329,28.93,24.36,12.67,9.95,24.17,836.86,2337,13.87,28.929058,A
2,19/12,11982,2523,2523,1794,1183,1198,10.75,107121,20062,87059,86506,0.0,9550,837,-501,-129,21.06,9.87,12.39,9.96,23.04,850.82,873,12.42,21.056585,B
3,20/3,11784,3150,3150,4156,2939,2930,9.00,108594,24862,83732,83170,562.0,9550,2336,-1392,-86,26.73,24.94,13.05,9.79,29.69,823.16,2134,9.71,26.731161,C
4,20/6,13188,3947,3947,4020,2935,2931,4.00,110282,23386,86896,86330,566.0,9550,7855,-214,-5501,29.93,22.25,12.20,9.40,26.91,837.40,2135,10.46,29.928723,A
